# Descripción del Proyecto

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import lightgbm as lgb
import plotly.graph_objs as go
import plotly.subplots as sp
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
from numpy.random import RandomState
import plotly.graph_objs as go
import plotly.express as px
import time

In [2]:
rs = RandomState(seed=1984)

In [3]:
df = pd.read_csv("/datasets/car_data.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Se puede observar que el dataframe contiene un total de 354,369 registros y 16 columnas donde se pueden identificar datos faltantes en las columnas VehicleType, Gearbox, Model, FuelType, NotRepaired y NumberOfPictures.

Para mejorar la coherencia, la legibilidad y la practicidad en el trabajo voy a cambiar los nombres de las columnas a minúsculas y a reescribir los títulos a continuación.

In [5]:
df = df.rename(columns={'DateCrawled': 'date_crawled', 'Price': 'price', 'VehicleType': 'vehicle_type', 'RegistrationYear': 'registration_year', 'Gearbox': 'gearbox', 'Power': 'power', 'Model': 'model', 'Mileage': 'mileage', 'RegistrationMonth': 'registration_month', 'FuelType': 'fuel_type', 'Brand': 'brand', 'NotRepaired': 'not_repaired', 'DateCreated': 'date_created', 'NumberOfPictures': 'number_of_pictures', 'PostalCode': 'postal_code', 'LastSeen': 'last_seen'})

# Columnas numéricas del dataset

In [6]:
df.describe()

,price,registration_year,power,mileage,registration_month,number_of_pictures,postal_code
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


# Valores no válidos en columnas numéricas

A la hora de analizar las columnas con valores numéricos pude identificar algunos valores que no son coherentes y se pueden considerar como datos erroneos como por ejemplo en año de registro hay valores atípicos como 1000 o 9999 que no corresponden a años válidos para el registro de autos, luego en mes de registro se observan filas con un valor de cero lo cual no es válido ya que el mes debe estar en el rango de 1 y 12, y así hay errores con otros datos por lo tanto abordaré este problema en los siguientes apartados para limpiar correctamente los datos y poder hacer un buen trabajo.

# Columnas numéricas eliminadas

Existen varias columnas numéricas que eliminaré en la sección de cambios debido a que considero no aportaban nada a nuestro objetivo y al contrario podrían perjudicar en algunos aspectos, por lo tanto estas columnas que eliminaré son: number_of_pictures, postal_code, date_crawled, date_created y last_seen.

# Columna registration_year

In [ ]:
df_filtered = df[(df['registration_year'] >= 1900) & (df['registration_year'] <= 2023)]

histogram = go.Histogram(x=df_filtered['registration_year'], nbinsx=50, name='Histograma')
boxplot = go.Box(x=df_filtered['registration_year'], name='Boxplot')

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Histograma", "Boxplot"))
fig.add_trace(histogram, row=1, col=1)
fig.add_trace(boxplot, row=1, col=2)

fig.update_layout(title='Análisis de la columna registration_year', showlegend=False)
fig.show()

Despues de hacer el análisis he tomado la decisión de eliminar los datos anteriores a 1989 y posteriores a 2019 en la siguiente sección del proyecto ya que considero presentan valores atípicos o incorrectos para el año de registro de un vehículo que pueden afectar la coherencia de los reslutados.

# Columna power

In [ ]:
hist = go.Histogram(x=df['power'], name='Potencia')

box = go.Box(y=df['power'], name='Potencia')

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Histograma', 'Boxplot'))
fig.add_trace(hist, row=1, col=1)
fig.add_trace(box, row=1, col=2)

fig.update_layout(width=900, height=400, title='Análisis de la columna power')

fig.show()

Después de observar los datos en este apartado, con el objetivo de mantener la coherencia y evitar valores extremos que puedan afectar el análisis y la precisión del modelo he decidido eliminar los datos de las filas cuyo valor en la columna power supere los 2,000 caballos de vapor todo esto en base a lo que considero datos realistas ya que cifras mas elevadas las considero poco probables y fiables.

# Columna mileage

In [ ]:
hist = go.Histogram(x=df['mileage'], name='Kilometraje')

box = go.Box(y=df['mileage'], name='Kilometraje')

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Histograma', 'Boxplot'))
fig.add_trace(hist, row=1, col=1)
fig.add_trace(box, row=1, col=2)

fig.update_layout(width=900, height=400, title='Análisis de la columna mileage')

fig.show()

En este apartado dejaré los datos como estan ya que según mi punto de vista no hay algun problema con ellos.

# Columna registration_month

In [ ]:
df["is_zero"] = (df["registration_month"] == 0)

fig = px.histogram(df, x="registration_month", nbins=24, color="is_zero")

fig.show()

Para mantener la coherencia y consistencia de los datos he decidido eliminar las filas que contengan el valor cero en la columna registration_month en el siguiente apartado ya que estas filas no aportan información válida y podrían afectar el análisis y los resultados del modelo.

# Cambios en columnas numéricas

In [11]:
df_copia = df.copy()

In [12]:
df = df.drop(['number_of_pictures', 'postal_code', 'date_crawled', 'date_created', 'last_seen'], axis=1)

df = df.loc[(df['registration_year'] >= 1988) & (df['registration_year'] <= 2020)]

df = df.loc[df['power'] < 2000]
   
df = df.loc[df['registration_month'] != 0]

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311147 entries, 1 to 354368
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               311147 non-null  int64 
 1   vehicle_type        288880 non-null  object
 2   registration_year   311147 non-null  int64 
 3   gearbox             301659 non-null  object
 4   power               311147 non-null  int64 
 5   model               298977 non-null  object
 6   mileage             311147 non-null  int64 
 7   registration_month  311147 non-null  int64 
 8   fuel_type           292510 non-null  object
 9   brand               311147 non-null  object
 10  not_repaired        263414 non-null  object
 11  is_zero             311147 non-null  bool  
dtypes: bool(1), int64(5), object(6)
memory usage: 28.8+ MB


# Columnas categóricas del dataset

En esta sección revisaré las columnas categóricas del dataset del proyecto en donde realizaré un análisis de cada una de estas para identificar patrones, valores únicos y posibles acciones a tomar en cada caso.

# Columna vehicle_type

In [13]:
print(df['vehicle_type'].value_counts())
print(df['vehicle_type'].isna().sum())

sedan          83616
small          72416
wagon          60177
bus            27025
convertible    18235
coupe          13789
suv            11092
other           2530
Name: vehicle_type, dtype: int64
22267


Después de  analisar esta columna he tomado la decisión de asignar los datos faltantes a la categoría other en la siguiente sección, esto porque me parece la mejor opción para manejar los valores ausentes en esta columna. 

# Columna gearbox

In [14]:
print(df['gearbox'].value_counts())
print(df['gearbox'].isna().sum())

manual    240794
auto       60865
Name: gearbox, dtype: int64
9488


Debido a que la categoría manual es la más frecuente en el dataset en comparación con la categoría automático he decidido imputar los valores faltantes en la categoría manual en la siguiente sección de cambios.

# Columna model

In [27]:
print(df['model'].value_counts())
print(df['model'].isna().sum())
print(df['model'].nunique())

golf                  25444
other                 19753
3er                   17811
polo                  11386
corsa                 11004
                      ...  
i3                        7
samara                    6
rangerover                3
range_rover_evoque        2
serie_2                   1
Name: model, Length: 248, dtype: int64
12170
248


Después de analizar la columna model y encontrar que hay 248 valores únicos en esta columna he decidido eliminarla del conjunto de datos con el propósito de evitar problemas de sobreajuste y mejorar la capacidad del modelo para generalizar a nuevos datos.

# Columna fuel_type

In [28]:
print(df['fuel_type'].value_counts())
print(df['fuel_type'].isna().sum())
print(df['fuel_type'].nunique())

petrol      193422
gasoline     93287
lpg           4841
cng            530
hybrid         225
other          124
electric        81
Name: fuel_type, dtype: int64
18637
7


Después de analizar la columna fuel_type y encontrar datos faltantes además de que hay 7 valores posibles en total para indicar el tipo de combustible utilizado por los autos, he decidido imputar los valores faltantes en la categoría other para conservar la información disponible en los registros y evitar la eliminación de datos importantes.

# Columna brand

In [29]:
print(df['brand'].value_counts())
print(df['brand'].isna().sum())
print(df['brand'].nunique())

volkswagen        66594
opel              34309
bmw               33259
mercedes_benz     28570
audi              26520
ford              21918
renault           15671
peugeot            9959
fiat               8187
seat               6282
skoda              5234
mazda              4998
smart              4858
citroen            4570
nissan             4418
toyota             4298
hyundai            3353
mini               3095
volvo              2849
mitsubishi         2659
honda              2465
kia                2312
suzuki             2063
alfa_romeo         1976
sonstige_autos     1727
chevrolet          1443
chrysler           1256
dacia               858
daihatsu            688
subaru              645
jeep                575
porsche             537
daewoo              490
saab                479
land_rover          466
jaguar              451
rover               403
lancia              392
lada                161
trabant             159
Name: brand, dtype: int64
0
40


Después de analizar la columna brand y encontrar 40 valores únicos, he decidido al igual que con la columna model eliminarla ya que considero que es la mejor opción en este caso y así poder evitar también problemas de sobreajuste.

# Columna not_repaired

In [30]:
print(df['not_repaired'].value_counts())
print(df['not_repaired'].isna().sum())
print(df['not_repaired'].nunique())

no     233207
yes     30207
Name: not_repaired, dtype: int64
47733
2


En esta columna encontré 47,733 valores ausentes y debido a que solo hay posiblidad de dos respuestas en este apartado (sí y no), he decidido asignar el valor de no a los registros faltantes ya que considero que los registros sin información disponible probablemente signifiquen que no han sido reparados todavía.

# Cambio en columnas categóricas

In [31]:
df['vehicle_type'].fillna(value='other', axis=0, inplace=True)

df['gearbox'].fillna(value='manual', axis=0, inplace=True)

df['fuel_type'].fillna(value='other', axis=0, inplace=True)

df['not_repaired'].fillna(value='no', axis=0, inplace=True)

In [32]:
df = df.drop(['model'], axis=1)
df = df.drop(['brand'], axis=1)

df = df.drop(['is_zero'], axis=1)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311147 entries, 1 to 354368
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               311147 non-null  int64 
 1   vehicle_type        311147 non-null  object
 2   registration_year   311147 non-null  int64 
 3   gearbox             311147 non-null  object
 4   power               311147 non-null  int64 
 5   mileage             311147 non-null  int64 
 6   registration_month  311147 non-null  int64 
 7   fuel_type           311147 non-null  object
 8   not_repaired        311147 non-null  object
dtypes: int64(5), object(4)
memory usage: 23.7+ MB


In [34]:
df_clean = df.copy()

# Datos con OrdinalEncoder

In [35]:
enc = OrdinalEncoder()
df_enc = df_clean.copy()
df_enc[['vehicle_type', 'gearbox', 'fuel_type', 'not_repaired']] = enc.fit_transform(df_clean[['vehicle_type', 'gearbox', 'fuel_type', 'not_repaired']])

df_ohe = pd.get_dummies(df_clean, columns=['vehicle_type', 'gearbox', 'fuel_type', 'not_repaired'], drop_first=True)

df_ohe.shape

(311147, 20)

Creación de features y target

In [36]:
rest_enc, valid = train_test_split(df_enc, test_size=0.2, random_state=rs)

train, test = train_test_split(rest_enc, test_size=0.25, random_state=rs)

features_train_enc = train.drop(['price'], axis=1)
target_train_enc = train['price']

features_test_enc = test.drop(['price'], axis=1)
target_test_enc = test['price']

features_valid_enc = valid.drop(['price'], axis=1)
target_valid_enc = valid['price'] 

features_rest_enc = rest_enc.drop(['price'], axis=1)
target_rest_enc = rest_enc['price']

print(features_train_enc.shape)
print(target_train_enc.shape)
print(features_test_enc.shape)
print(target_test_enc.shape)
print(features_valid_enc.shape)
print(target_valid_enc.shape)

(186687, 8)
(186687,)
(62230, 8)
(62230,)
(62230, 8)
(62230,)


# Datos con OneHotEncoder

In [37]:
rest_ohe, valid = train_test_split(df_ohe, test_size=0.2, random_state=rs)

train, test = train_test_split(rest_ohe, test_size=0.25, random_state=rs)

features_train_ohe = train.drop(['price'], axis=1)
target_train_ohe = train['price']

features_test_ohe = test.drop(['price'], axis=1)
target_test_ohe = test['price']

features_valid_ohe = valid.drop(['price'], axis=1)
target_valid_ohe = valid['price']

features_rest_ohe = rest_ohe.drop(['price'], axis=1)
target_rest_ohe = rest_ohe['price']

print(features_train_ohe.shape)
print(target_train_ohe.shape)
print(features_test_ohe.shape)
print(target_test_ohe.shape)
print(features_valid_ohe.shape)
print(target_valid_ohe.shape)

(186687, 19)
(186687,)
(62230, 19)
(62230,)
(62230, 19)
(62230,)


# Datos sin codificar

In [38]:
rest, valid = train_test_split(df, test_size=0.2, random_state=rs)

train, test = train_test_split(rest, test_size=0.25, random_state=rs)

features_train = train.drop(['price'], axis=1)
target_train = train['price']

features_test = test.drop(['price'], axis=1)
target_test = test['price']

features_valid = valid.drop(['price'], axis=1)
target_valid = valid['price']

features_rest = rest.drop(['price'], axis=1)
target_rest = rest['price']

print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)

(186687, 8)
(186687,)
(62230, 8)
(62230,)
(62230, 8)
(62230,)


# Entrenamiento del modelo

# Regresión lineal

In [ ]:
reg = LinearRegression()

start_time = time.time()
reg.fit(features_train_ohe, target_train_ohe)
train_time = time.time() - start_time

start_time = time.time()
reg_pred_ohe = reg.predict(features_valid_ohe)
pred_time = time.time() - start_time

rmse = mean_squared_error(target_valid_ohe, reg_pred_ohe, squared=False)

print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')
print(f'Tiempo de predicción: {pred_time:.2f} segundos')
print(f'RECM: {rmse:.2f}')

# Bosque aleatorio con ajuste de parámetros

In [ ]:
best_score = float('inf')
n = 1

for roots in range(3, 8):
    for leafs in range(3, 8):
        print(f'Ronda {n}')
        
        forest = RandomForestRegressor(random_state=rs, max_depth=roots, min_samples_leaf=leafs)
        start_time = time.time()
        forest.fit(features_train_enc, target_train_enc)
        train_time = time.time() - start_time
        
        start_time = time.time()
        f_pred = forest.predict(features_valid_enc)
        pred_time = time.time() - start_time
        
        rmse = mean_squared_error(target_valid_enc, f_pred, squared=False)
        print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')
        print(f'Tiempo de predicción: {pred_time:.2f} segundos')
        print(f'RECM: {rmse:.2f}')
        print()
        
        if rmse < best_score:
            best_score = rmse
            best_root = roots
            best_leaf = leafs
            best_pred = f_pred
            best_n = n
                        
        n += 1

best_forest = RandomForestRegressor(random_state=rs, max_depth=best_root, min_samples_leaf=best_leaf)

print(f'Mejor ronda: {best_n}')
print(f'Mejor RECM: {best_score:.2f}')

# CatBoost

In [ ]:
cat = CatBoostRegressor(iterations=200, cat_features=['vehicle_type', 'fuel_type', 'not_repaired', 'gearbox'])
start_time = time.time()
cat.fit(features_train, target_train, eval_set=(features_valid, target_valid))
train_time = time.time() - start_time

start_time = time.time()
cat_pred = cat.predict(features_valid)
pred_time = time.time() - start_time

rmse = mean_squared_error(target_valid, cat_pred, squared=False)

print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')
print(f'Tiempo de predicción: {pred_time:.2f} segundos')
print(f'RECM: {rmse:.2f}')

# LightGBM con ajuste de hiperparámetros

In [ ]:
def convert_columns_to_category(df, columns):
    for column in columns:
        df[column] = df[column].astype('category')
    return df

categorical_columns = ['vehicle_type', 'fuel_type', 'not_repaired', 'gearbox']
features_train = convert_columns_to_category(features_train, categorical_columns)
features_valid = convert_columns_to_category(features_valid, categorical_columns)
features_test = convert_columns_to_category(features_test, categorical_columns)
features_rest = convert_columns_to_category(features_rest, categorical_columns)

lgbm = lgb.LGBMRegressor(random_state=rs, learning_rate=0.05, n_estimators=500,
                         max_depth=5, num_leaves=31, min_child_samples=20,
                         subsample=0.8, colsample_bytree=0.8)

start_time = time.time()
lgbm.fit(features_train, target_train, eval_metric='RMSE',
         categorical_feature=categorical_columns, eval_set=(features_valid, target_valid))
train_time = time.time() - start_time

start_time = time.time()
lgbm_pred = lgbm.predict(features_valid)
pred_time = time.time() - start_time

rmse = mean_squared_error(target_valid, lgbm_pred, squared=False)

print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')
print(f'Tiempo de predicción: {pred_time:.2f} segundos')
print(f'RECM: {rmse:.2f}')

## Análisis del modelo

Para saber que modelo se adecúa mejor a nuestro propósito debemos tomar en cuenta el rendimiento y el tiempo de computación, por eso viendo en retrospectiva y comparando cada uno de los modelos se puede observar que en bosque aleatorio el RECM es el segundo puntaje más alto pero el tiempo de entrenamiento es largo, en la regresión lineal el tiempo de entrenamiento es el más bajo pero tiene el RECM más alto por lo cual es el modelo más impreciso, luego el modelo de CatBoost es el más preciso pero es muy lento el entrenamiento mientras que con LightGBM el RECM es ligeramente superior a CatBoost pero el tiempo de entrenamiento es bastante mejor por eso considero que LightGBM es el mejor modelo para nuestro propósito ya que considero que el tiempo de entrenamiento más rápido es más importante que la muy ligera variasíon en el RECM.


A continuación haré el entrenamiento con los datos de validación con el modelo LigthGMB:

In [32]:
lgbm = lgb.LGBMRegressor(random_state=rs)

start_time = time.time()
lgbm.fit(features_rest, target_rest, eval_metric = 'RMSE', 
         categorical_feature=['vehicle_type', 'fuel_type', 'not_repaired', 'gearbox'], 
         eval_set=(features_test, target_test))
train_time = time.time() - start_time

print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')

start_time = time.time()
lgbm_pred = lgbm.predict(features_test)
pred_time = time.time() - start_time

print(f'Tiempo de predicción: {pred_time:.2f} segundos')

rmse = mean_squared_error(target_test, lgbm_pred, squared=False)
print(f'RECM: {rmse:.2f}')

/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning:

Using categorical_feature in Dataset.

/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning:

categorical_feature in Dataset is overridden.
New categorical_feature is ['fuel_type', 'gearbox', 'not_repaired', 'vehicle_type']

/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning:

Overriding the parameters from Reference Dataset.

/opt/conda/envs/python3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning:

categorical_column in param dict is overridden.



[1]	valid_0's rmse: 4241.63	valid_0's l2: 1.79914e+07
[2]	valid_0's rmse: 3957.93	valid_0's l2: 1.56652e+07
[3]	valid_0's rmse: 3710.92	valid_0's l2: 1.37709e+07
[4]	valid_0's rmse: 3489.42	valid_0's l2: 1.2176e+07
[5]	valid_0's rmse: 3298.87	valid_0's l2: 1.08825e+07
[6]	valid_0's rmse: 3135.63	valid_0's l2: 9.83218e+06
[7]	valid_0's rmse: 2990.74	valid_0's l2: 8.9445e+06
[8]	valid_0's rmse: 2865.51	valid_0's l2: 8.21115e+06
[9]	valid_0's rmse: 2757.1	valid_0's l2: 7.60161e+06
[10]	valid_0's rmse: 2664.85	valid_0's l2: 7.1014e+06
[11]	valid_0's rmse: 2581.47	valid_0's l2: 6.66397e+06
[12]	valid_0's rmse: 2508.86	valid_0's l2: 6.2944e+06
[13]	valid_0's rmse: 2447.89	valid_0's l2: 5.99219e+06
[14]	valid_0's rmse: 2394.33	valid_0's l2: 5.73282e+06
[15]	valid_0's rmse: 2348.26	valid_0's l2: 5.51431e+06
[16]	valid_0's rmse: 2308.58	valid_0's l2: 5.32955e+06
[17]	valid_0's rmse: 2273.39	valid_0's l2: 5.16831e+06
[18]	valid_0's rmse: 2243.22	valid_0's l2: 5.03206e+06
[19]	valid_0's rmse: 221

Como se puede apreciar estos resultados indican que el modelo LightGBM logró mejorar tanto los tiempos de entrenamiento y predicción como la precisión con los datos de validación ya que el tiempo de entrenamiento y el tiempo de predicción disminuyeron considerablemente, además de que el RECM se redujo, de esta forma y como conclusión considero que el modelo LightGBM resultó ser la mejor opción en comparación con los otros modelos evaluados.

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos